Jupyter-Notebook zum Testen des stochastischen Lastmodells

In [ ]:
###IMPORT
import numpy as np
import pandas as pd
import dataprocessing
from stochasticmodel import StochasticLoadModel, StochasticSimulationResult
import loaddata
import matplotlib.pyplot as plt
import seaborn as sns

%reload_ext autoreload
%autoreload 2

Daten laden

In [ ]:
stationen=loaddata.load_stations()
display(stationen)

In [ ]:
list_power_d, list_power_yd, list_power_ysd, list_power_ymd, list_mat_power_d, list_power_ywd, list_power_yswd, list_power_only_d = loaddata.load_data_from_measurements([0,1,2,3,4,5,6,7,8], d=True, yd=True, ysd=True,ywd=True,yswd=True)

Stochastische Simulation

In [ ]:
MAPES=np.zeros([9,3])
all_sim=list()
all_ref=list()
all_res=list()
all_mape=list()

for station in range(0,9):
    station_no = station
    power_d = list_power_d[station_no]
    power_yd = list_power_yd[station_no]
    power_ysd = list_power_ysd[station_no]
    power_ywd = list_power_ywd[station_no]
    power_yswd = list_power_yswd[station_no]
    data_input = [power_yswd[2018], power_yd[2018],power_yswd[2018]]
    reference = power_yd[2018]
    mymodel=StochasticLoadModel(normalization=[4,1,1,2],order_regression=6)
    mymodel.parameterize(data_in=data_input,plots=False)
    res = mymodel.simulate(n_runs=10)
    energy_reference = 0.25*reference.sum().sum()
    denormalized_result=mymodel.denormalize(res,new_case_4=4,energy=energy_reference,max_value=reference.max().max())
    A, B, C, D = denormalized_result.analyze(reference,plots=False,show_tables=False, export_plots=False)
    MAPES[station,:]=A
    all_res.insert(len(all_res),denormalized_result)
    all_ref.insert(len(all_ref),B)
    all_sim.insert(len(all_sim),C)
    all_mape.insert(len(all_mape),D)